In [1]:
from mlagents_envs.environment import ActionTuple, UnityEnvironment, BaseEnv
import numpy as np
import torch
import copy

In [2]:
# GridBoard
import numpy as np
import random
import sys

def randPair(s,e):
    return np.random.randint(s,e), np.random.randint(s,e)

class BoardPiece:

    def __init__(self, name, code, pos):
        self.name = name #name of the piece
        self.code = code #an ASCII character to display on the board
        self.pos = pos #2-tuple e.g. (1,4)

class BoardMask:

    def __init__(self, name, mask, code):
        self.name = name
        self.mask = mask
        self.code = code

    def get_positions(self): #returns tuple of arrays
        return np.nonzero(self.mask)

def zip_positions2d(positions): #positions is tuple of two arrays
    x,y = positions
    return list(zip(x,y))

class GridBoard:

    def __init__(self, size=4):
        self.size = size #Board dimensions, e.g. 4 x 4
        self.components = {} #name : board piece
        self.masks = {}

    def addPiece(self, name, code, pos=(0,0)):
        newPiece = BoardPiece(name, code, pos)
        self.components[name] = newPiece

    #basically a set of boundary elements
    def addMask(self, name, mask, code):
        #mask is a 2D-numpy array with 1s where the boundary elements are
        newMask = BoardMask(name, mask, code)
        self.masks[name] = newMask

    def movePiece(self, name, pos):
        move = True
        for _, mask in self.masks.items():
            if pos in zip_positions2d(mask.get_positions()):
                move = False
        if move:
            self.components[name].pos = pos

    def delPiece(self, name):
        del self.components['name']

    def render(self):
        dtype = '<U2'
        displ_board = np.zeros((self.size, self.size), dtype=dtype)
        displ_board[:] = ' '

        for name, piece in self.components.items():
            displ_board[piece.pos] = piece.code

        for name, mask in self.masks.items():
            displ_board[mask.get_positions()] = mask.code

        return displ_board

    def render_np(self):
        num_pieces = len(self.components) + len(self.masks)
        displ_board = np.zeros((num_pieces, self.size, self.size), dtype=np.uint8)
        layer = 0
        for name, piece in self.components.items():
            pos = (layer,) + piece.pos
            displ_board[pos] = 1
            layer += 1

        for name, mask in self.masks.items():
            x,y = self.masks['boundary'].get_positions()
            z = np.repeat(layer,len(x))
            a = (z,x,y)
            displ_board[a] = 1
            layer += 1
        return displ_board

def addTuple(a,b):
    return tuple([sum(x) for x in zip(a,b)])

In [3]:
class Gridworld:

    def __init__(self, size=4, mode='static'):
        if size >= 4:
            self.board = GridBoard(size=size)
        else:
            print("Minimum board size is 4. Initialized to size 4.")
            self.board = GridBoard(size=4)

        #Add pieces, positions will be updated later
        self.board.addPiece('Player','P',(0,0))
        self.board.addPiece('Goal','+',(1,0))
        self.board.addPiece('Pit','-',(2,0))
        self.board.addPiece('Wall','W',(3,0))

        if mode == 'static':
            self.initGridStatic()
        elif mode == 'player':
            self.initGridPlayer()
        else:
            self.initGridRand()

    #Initialize stationary grid, all items are placed deterministically
    def initGridStatic(self):
        #Setup static pieces
        self.board.components['Player'].pos = (0,3) #Row, Column
        self.board.components['Goal'].pos = (0,0)
        self.board.components['Pit'].pos = (0,1)
        self.board.components['Wall'].pos = (1,1)

    #Check if board is initialized appropriately (no overlapping pieces)
    #also remove impossible-to-win boards
    def validateBoard(self):
        valid = True

        player = self.board.components['Player']
        goal = self.board.components['Goal']
        wall = self.board.components['Wall']
        pit = self.board.components['Pit']

        all_positions = [piece for name,piece in self.board.components.items()]
        all_positions = [player.pos, goal.pos, wall.pos, pit.pos]
        if len(all_positions) > len(set(all_positions)):
            return False

        corners = [(0,0),(0,self.board.size), (self.board.size,0), (self.board.size,self.board.size)]
        #if player is in corner, can it move? if goal is in corner, is it blocked?
        if player.pos in corners or goal.pos in corners:
            val_move_pl = [self.validateMove('Player', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            val_move_go = [self.validateMove('Goal', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            if 0 not in val_move_pl or 0 not in val_move_go:
                #print(self.display())
                #print("Invalid board. Re-initializing...")
                valid = False

        return valid

    #Initialize player in random location, but keep wall, goal and pit stationary
    def initGridPlayer(self):
        #height x width x depth (number of pieces)
        self.initGridStatic()
        #place player
        self.board.components['Player'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridPlayer()

    #Initialize grid so that goal, pit, wall, player are all randomly placed
    def initGridRand(self):
        #height x width x depth (number of pieces)
        self.board.components['Player'].pos = randPair(0,self.board.size)
        self.board.components['Goal'].pos = randPair(0,self.board.size)
        self.board.components['Pit'].pos = randPair(0,self.board.size)
        self.board.components['Wall'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridRand()

    def validateMove(self, piece, addpos=(0,0)):
        outcome = 0 #0 is valid, 1 invalid, 2 lost game
        pit = self.board.components['Pit'].pos
        wall = self.board.components['Wall'].pos
        new_pos = addTuple(self.board.components[piece].pos, addpos)
        if new_pos == wall:
            outcome = 1 #block move, player can't move to wall
        elif max(new_pos) > (self.board.size-1):    #if outside bounds of board
            outcome = 1
        elif min(new_pos) < 0: #if outside bounds
            outcome = 1
        elif new_pos == pit:
            outcome = 2

        return outcome

    def makeMove(self, action):
        #need to determine what object (if any) is in the new grid spot the player is moving to
        #actions in {u,d,l,r}
        def checkMove(addpos):
            if self.validateMove('Player', addpos) in [0,2]:
                new_pos = addTuple(self.board.components['Player'].pos, addpos)
                self.board.movePiece('Player', new_pos)

        if action == 'u': #up
            checkMove((-1,0))
        elif action == 'd': #down
            checkMove((1,0))
        elif action == 'l': #left
            checkMove((0,-1))
        elif action == 'r': #right
            checkMove((0,1))
        else:
            pass

    def reward(self):
        if (self.board.components['Player'].pos == self.board.components['Pit'].pos):
            return -10
        elif (self.board.components['Player'].pos == self.board.components['Goal'].pos):
            return 10
        else:
            return -1

    def display(self):
        return self.board.render()

In [4]:
action_set = {
  0: 'u',
  1: 'd',
  2: 'l',
  3: 'r',
}

In [127]:
l1 = 4*4*4
l2 = 150
l3 = 100
l4 = 4

model = torch.nn.Sequential(
  torch.nn.Linear(l1, l2),
  torch.nn.ReLU(),
  torch.nn.Linear(l2,l3),
  torch.nn.ReLU(),
  torch.nn.Linear(l3,l4),
  torch.nn.Softmax(dim=1)
)
  
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [128]:
def discount_rewards(rewards: np.ndarray, gamma):
    reversed = np.copy(rewards)[::-1]
    discounted_rewards = []
    for i, reward in enumerate(reversed):
        discounted_rewards.append(reward + (0 if i == 0 else reversed[i - 1]))
        reversed[i] = reward * gamma
        if i > 0:
            reversed[i] += reversed[i - 1] * gamma
    return np.array(discounted_rewards[::-1])

In [129]:
def loss_fn(predictions, rewards):
    return -1 * torch.sum(rewards * torch.log(predictions))

In [137]:
def get_trajectories(model, max_step):
    states = []
    actions = []
    action_sets = []
    rewards = []

    env = Gridworld(size=4, mode='random')
    for i in range(max_step):
        state = env.board.render_np().reshape(64)

        states.append(state)
        pred = model(torch.from_numpy(np.array([state])).float())
        action = np.random.choice(np.array([0, 1, 2, 3]), p=pred.detach().numpy().flatten())
        actions.append(action)
        action = action_set[action]
        action_sets.append(action)

        env.makeMove(action)
        
        reward = env.reward()
        rewards.append(float(reward))
        
        if reward != -1:
            if reward > 0:
                print("Game won! Reward: %s" % (reward,))
                break
            else:
                print("Game LOST. Reward: %s" % (reward,))
                break

    return torch.from_numpy(np.array(states)).float(), np.array(actions), np.array(rewards), np.array(action_sets)

In [138]:
EPOCH = 10000
GAMMA = 0.9
MAX_STEP = 50

for i in range(EPOCH):
    states, actions, rewards, action_sets = get_trajectories(model, MAX_STEP)
    predictions = model(states)
    cumulative_reward = np.sum(rewards)
    print(f'EPOCH {i}: cumulative reward {cumulative_reward}')
    discounted_rewards = torch.tensor(discount_rewards(rewards, GAMMA))

    actions = torch.tensor(actions.reshape(-1, 1)).long()
    prob_batch = predictions.gather(dim=1,index=actions).squeeze()
    loss = loss_fn(prob_batch, discounted_rewards)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Game LOST. Reward: -10
EPOCH 0: cumulative reward -10.0
Game won! Reward: 10
EPOCH 1: cumulative reward -2.0
Game LOST. Reward: -10
EPOCH 2: cumulative reward -18.0
Game won! Reward: 10
EPOCH 3: cumulative reward 9.0
Game LOST. Reward: -10
EPOCH 4: cumulative reward -11.0
Game LOST. Reward: -10
EPOCH 5: cumulative reward -38.0
Game won! Reward: 10
EPOCH 6: cumulative reward 10.0
Game won! Reward: 10
EPOCH 7: cumulative reward -1.0
Game won! Reward: 10
EPOCH 8: cumulative reward 1.0
Game won! Reward: 10
EPOCH 9: cumulative reward 3.0
Game won! Reward: 10
EPOCH 10: cumulative reward 10.0
Game LOST. Reward: -10
EPOCH 11: cumulative reward -20.0
Game won! Reward: 10
EPOCH 12: cumulative reward 0.0
Game LOST. Reward: -10
EPOCH 13: cumulative reward -26.0
Game won! Reward: 10
EPOCH 14: cumulative reward -1.0
Game LOST. Reward: -10
EPOCH 15: cumulative reward -14.0
Game won! Reward: 10
EPOCH 16: cumulative reward 3.0
Game LOST. Reward: -10
EPOCH 17: cumulative reward -11.0
Game LOST. Reward: 

In [143]:
def test_model(model, mode='random', display=True):
  i = 0
  test_game = Gridworld(mode=mode)
  state_ = test_game.board.render_np().reshape(1,64)
  state = torch.from_numpy(state_).float()
  if display:
    print("Initial State:")
    print(test_game.display())
  status = 1
  while(status == 1): 
    qval = model(state)
    qval_ = qval.data.numpy()
    action_ = np.argmax(qval_) 
    action = action_set[action_]
    if display:
      print(qval)
      print('Move #: %s; Taking action: %s' % (i, action))
    test_game.makeMove(action)
    state_ = test_game.board.render_np().reshape(1,64)
    state = torch.from_numpy(state_).float()
    if display:
      print(test_game.display())
    reward = test_game.reward()
    if reward != -1:
      if reward > 0:
        status = 2
        if display:
          print("Game won! Reward: %s" % (reward,))
      else:
        status = 0
        if display:
          print("Game LOST. Reward: %s" % (reward,))
    i += 1
    if (i > 15):
      if display:
        print("Game lost; too many moves.")
      break
  win = True if status == 2 else False
  return win

test_model(model)

Initial State:
[[' ' ' ' ' ' '+']
 [' ' ' ' ' ' ' ']
 [' ' ' ' '-' 'W']
 [' ' 'P' ' ' ' ']]
tensor([[9.2150e-01, 4.7179e-10, 3.9263e-09, 7.8498e-02]],
       grad_fn=<SoftmaxBackward>)
Move #: 0; Taking action: u
[[' ' ' ' ' ' '+']
 [' ' ' ' ' ' ' ']
 [' ' 'P' '-' 'W']
 [' ' ' ' ' ' ' ']]
tensor([[9.8758e-01, 5.8247e-10, 5.5657e-08, 1.2419e-02]],
       grad_fn=<SoftmaxBackward>)
Move #: 1; Taking action: u
[[' ' ' ' ' ' '+']
 [' ' 'P' ' ' ' ']
 [' ' ' ' '-' 'W']
 [' ' ' ' ' ' ' ']]
tensor([[4.8903e-01, 9.6206e-08, 2.9410e-08, 5.1097e-01]],
       grad_fn=<SoftmaxBackward>)
Move #: 2; Taking action: r
[[' ' ' ' ' ' '+']
 [' ' ' ' 'P' ' ']
 [' ' ' ' '-' 'W']
 [' ' ' ' ' ' ' ']]
tensor([[9.8733e-01, 2.5501e-08, 4.0000e-06, 1.2665e-02]],
       grad_fn=<SoftmaxBackward>)
Move #: 3; Taking action: u
[[' ' ' ' 'P' '+']
 [' ' ' ' ' ' ' ']
 [' ' ' ' '-' 'W']
 [' ' ' ' ' ' ' ']]
tensor([[4.3468e-03, 1.0691e-03, 4.5882e-06, 9.9458e-01]],
       grad_fn=<SoftmaxBackward>)
Move #: 4; Taking actio

True

In [144]:
max_games = 1000
wins = 0
for i in range(max_games):
    win = test_model(model, mode='random', display=False)
    if win:
        wins += 1
win_perc = float(wins) / float(max_games)
print("Games played: {0}, # of wins: {1}".format(max_games,wins))
print("Win percentage: {}%".format(100.0*win_perc))

Games played: 1000, # of wins: 809
Win percentage: 80.9%


==========================================